# True firing rates

Repeat for each layer

1. calculate spontaneous model's median fr. from l_i_ (NS_median)  
2. calculate in vivo model's median fr. from l_i: (M_median)  
3. calculate ground truth (GT) median FR from l_i: GT_median  
4. calculate true in vivo:  
  * median: GT_median + (M_median - NS_median)  
  * variance: GT_var + (M_var - NS_var)  

## Setup

In [11]:
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import numpy as np
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

# SET PROJECT PATH

PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/preprint_2023"
os.chdir(PROJ_PATH)
from src.nodes.utils import get_config, savefig
from src.nodes.validation import firing_rate as fr
from src.nodes.analysis.npx_probe import true_fr
from src.nodes.postpro import metadata as meta
import spikeinterface as si
import shutil
from src.nodes.postpro import (
    cell_type,
)
# SETUP EXPERIMENTS

# data
cfg_nv, _ = get_config("vivo_marques", "c26").values()
SORTED_PATH_nv = cfg_nv["sorting"]["sorters"]["kilosort4"]["10m"]["output"]

# model biophy. spontaneous [done]
cfg_ns, _ = get_config("silico_neuropixels", "concatenated").values()
SORTED_PATH_ns = cfg_ns["sorting"]["sorters"]["kilosort4"]["10m"]["output"]
GT_PATH_ns = cfg_ns["sorting"]["simulation"]["ground_truth"]["output"]
GT_PATH_ns_10m = cfg_ns["sorting"]["simulation"]["ground_truth"]["10m"]["output"]
                
# PARAMETERS
SINGLE_UNIT = True
layers = ["L1", "L2/3", "L4", "L5", "L6"]

# SETUP PLOT 

# set lognormal plot parameters
log_x_min = -3.0
log_x_max = 3.0
nbins = 30

# IF you don't want to use the feature, set the value to 1E100
# t_dec = 3600 # seconds until experimenter decides a cell is silent
t_dec = 1^100 # seconds until experimenter decides a cell is silent

cl = dict()
cl["COLOR_NV"] = [0.6, 0.6, 0.6]
cl["COLOR_NS"] = [0.9, 0.14, 0.15]
cl["COLOR_NB"] = [0.22, 0.5, 0.72] # blue
cl["COLOR_NE"] = [1, 0.49, 0] # orange
cl["COLOR_HV"] = "k" # black
cl["COLOR_HS"] = np.array([26, 152, 80]) / 255 # green

# axes
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 6  # 5-7 with Nature neuroscience as reference
plt.rcParams["lines.linewidth"] = 0.5 # typically between 0.5 and 1
plt.rcParams["axes.linewidth"] = 0.5 #1
plt.rcParams["axes.spines.top"] = False
plt.rcParams["xtick.major.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["xtick.minor.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["ytick.major.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["ytick.minor.width"] = 0.5 #0.8 #* 1.3
plt.rcParams["xtick.major.size"] = 3.5 * 1.1
plt.rcParams["xtick.minor.size"] = 2 * 1.1
plt.rcParams["ytick.major.size"] = 3.5 * 1.1
plt.rcParams["ytick.minor.size"] = 2 * 1.1
# legend
legend_cfg = {"frameon": False, "handletextpad": 0.5}
tight_layout_cfg = {"pad": 0.001}
LG_FRAMEON = False              # no legend frame

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2024-08-21 15:13:28,948 - root - utils.py - get_config - INFO - Reading experiment config.
2024-08-21 15:13:29,011 - root - utils.py - get_config - INFO - Reading experiment config. - done
2024-08-21 15:13:29,016 - root - utils.py - get_config - INFO - Reading experiment config.
2024-08-21 15:13:29,071 - root - utils.py - get_config - INFO - Reading experiment config. - done


In [12]:
def save_SortingTrue_metadata(gt_path, blueconfig_path, duration, save):

    # Get ground truth SortingExtractor
    # - get first 10 minutes
    SortingTrue = si.load_extractor(gt_path)
    SortingTrue = SortingTrue.frame_slice(
        start_frame=0, end_frame=duration * SortingTrue.sampling_frequency
    )

    # set unit features metadata (includes layer)
    SortingTrue = cell_type.label_true_cell_properties(
        SortingTrue, blueconfig_path, gt_path, save=False
    )

    # set unit firing rate metadata
    SortingTrue = meta.set_firing_rates(SortingTrue, duration, gt_path, save=False)

    # save
    if save:
        shutil.rmtree(GT_PATH_ns_10m, ignore_errors=True)
        SortingTrue.save(folder=GT_PATH_ns_10m)
    return SortingTrue

## Get metadata

In [13]:
# get sorted unit metadata
df_nv = fr.get_sorted_unit_meta(SORTED_PATH_nv)
df_ns = fr.get_sorted_unit_meta(SORTED_PATH_ns)

# keep units in cortex
df_nv = df_nv[np.isin(df_nv["layer"], ["L1", "L2/3", "L4", "L5", "L6"])]
df_ns = df_ns[np.isin(df_ns["layer"], ["L1", "L2/3", "L4", "L5", "L6"])]

# unit-test: check all in cortex
assert all(
    np.isin(np.sort(df_nv["layer"].unique()), ["L1", "L2/3", "L4", "L5", "L6"])
), "nor all in cortex"
assert all(
    np.isin(np.sort(df_ns["layer"].unique()), ["L1", "L2/3", "L4", "L5", "L6"])
), "nor all in cortex"

In [14]:
# get ground truth metadata (and save)
SortingTrue = save_SortingTrue_metadata(
    GT_PATH_ns, cfg_ns["dataeng"]["blueconfig"], 10 * 60, save=True
)
df_gt_ns = meta.get_gt_unit_meta_df(GT_PATH_ns_10m)

# unit-test: check all in cortex
assert all(
    np.isin(np.sort(df_gt_ns["layer"].unique()), ["L1", "L2/3", "L4", "L5", "L6"])
), "nor all in cortex"

### Curation or not

In [15]:
# curate units
if SINGLE_UNIT:
    df_nv = df_nv[df_nv["kslabel"] == "good"]
    df_ns = df_ns[df_ns["kslabel"] == "good"]

### Model's ground T. firing razes

In [6]:
# def get_true_unit_meta(sorted_path: str):
#     """_summary_

#     Returns:
#         pd.DataFrame: _description_
#     """
#     # load Sorting extractor
#     Sorting = si.load_extractor(sorted_path)

#     # record
#     unit_id_all = Sorting.unit_ids.tolist()
#     firing_rate_all = Sorting.get_property("firing_rates").astype(np.float32).tolist()
#     layer_all = Sorting.get_property("layer").tolist()
#     # layer_all = standardize_layers(layer_all)n
#     # KSLabel_all = get_kslabel(Sorting)
#     # amplitude_all = get_amplitude(Sorting)

#     # store in dataframe
#     return pd.DataFrame(
#         np.array(
#             [
#                 layer_all,
#                 firing_rate_all,
#                 # KSLabel_all,
#                 # amplitude_all,
#             ]
#         ).T,
#         index=unit_id_all,
#         columns=[
#             "layer",
#             "firing_rate",
#             "kslabel",
#             "amplitude",
#         ],
#     )

* Sorted unit firing rates were calculated from the 10 first minutes of recording. We do the same for the ground truth.

### Save dataset for Michael

In [7]:
layers = ["L1", "L2/3", "L4", "L5", "L6"]

# log10
df_log_fr = true_fr.get_log10fr_df(df_nv, df_ns, df_gt_ns, layers, log=True)
df_log_fr.to_csv(
    "/gpfs/bbp.cscs.ch/project/proj85/laquitai/spikebias_paper/true_firing_rate/log10_fr.csv"
)

# raw
df_raw_fr = true_fr.get_log10fr_df(df_nv, df_ns, df_gt_ns, layers, log=False)
df_raw_fr.to_csv(
    "/gpfs/bbp.cscs.ch/project/proj85/laquitai/spikebias_paper/true_firing_rate/raw_fr.csv"
)

In [10]:
display(df_log_fr)
display(df_raw_fr)

,firing rate,experiment,layer
283,-1.346787,GT,L1
279,-2.000000,GT,L1
280,0.259275,GT,L1
281,-2.477121,GT,L1
282,-0.273001,GT,L1
...,...,...,...
1253,-0.926893,GT,L6
1254,-2.778151,GT,L6
1255,-1.574031,GT,L6
1248,-0.133713,GT,L6


,firing rate,experiment,layer
283,0.045000,GT,L1
279,0.010000,GT,L1
280,1.816667,GT,L1
281,0.003333,GT,L1
282,0.533333,GT,L1
...,...,...,...
1253,0.118333,GT,L6
1254,0.001667,GT,L6
1255,0.026667,GT,L6
1248,0.735000,GT,L6


In [9]:
import pandas as pd

df_raw = pd.read_csv(
    "/gpfs/bbp.cscs.ch/project/proj85/laquitai/spikebias_paper/true_firing_rate/raw_fr.csv"
)

,Unnamed: 0,firing rate,experiment,layer
0,283,0.045000,GT,L1
1,279,0.010000,GT,L1
2,280,1.816667,GT,L1
3,281,0.003333,GT,L1
4,282,0.533333,GT,L1
...,...,...,...,...
1662,1253,0.118333,GT,L6
1663,1254,0.001667,GT,L6
1664,1255,0.026667,GT,L6
1665,1248,0.735000,GT,L6


In [18]:
# unit-testing
assert sum(df_log_fr["experiment"] == "GT") == sum(df_raw_fr["experiment"] == "GT")
assert sum(df_log_fr["experiment"] == "M") == sum(df_raw_fr["experiment"] == "M")